**Importing Libraries**

In [48]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

In [29]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Loading all review files**

In [2]:
from google.colab import files
files.upload()

Saving reviews_data_dump.zip to reviews_data_dump.zip


{'reviews_data_dump.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00!YeXG\x82<\t4\x01\x00\x00\x96\x01\x00\x00\x1d\x00\x00\x00reviews_badminton/credits.txt-\x90\xc1n\xc20\x10D\xcf\x8d\x94\x7f\xd8\x1f\x08\xa9\x84P\xa5HUU\nA\xad \xb4\x84\xb4\xe5\xb8$&6\xc4v\xe4\xb5I\xfd\xf7uZn\xab\xd5\xce\x9b\x9dY\xa0E(k\x83}\xcf\x1a\xc8\x8d\x96\x19\xe4\x9d\xe8/h,|\xb1#\t\xcb\xe2\xe8\xc5\x91\xd5\x92\x19\xd8\xb1\xab`\x03A\xae\r\xbc\x1b\xdd\xb8\xdafP\xf6\xdaX\x82\x04\x0e\xdbb\xf9\r\x9b\xe7\xcf\xd7\x12\xa6\xb3{(|\xa7\x15\x94\xdcY\xdb\x05J\x1c\xdd\x80\xb0\x16\xea\x92\x01\xb7\xb6\xa7,M\x87a\x98\x9cn\x9e\x93Z\xcb\xd4k\xc5~\x12\x89WAI\xe0$j\xe4$\xf4\xcfI<\xeb:=\xa4}*\xac<\xd5\xe7\xc6s\xd5\xf2\x93g\xfe\xa9\x17\xcdc\xb9_/\xf3\xb7\x87*\xffX\xed\xaaj7\x1dmC\x00\xc3\x1aa)\x8e\xf6\x0c%\x14N\x1e\x99\xc9`\x16G\xa5;JA$\xc2\x9f\x8b\xe0\ns\x9fA%Q"g\xc4\xe1\xc0T\x18;\x1c\x11[\xcbC\x01\x7f\xfa\r\x1b\xf5\x94\xc5\xd1\xcaY\xee:\x849\xc7\xd6#\x19\x16b\xbe\x1bTx\x1dW\x03\xa1\x89\xa3\x02[\x0c\xba3\xd2 \x94\x809v*\\\xad\xb1\xe5Nb(\xde\xc

**Unzip the dataset**

In [3]:
import zipfile
import os

In [4]:
zip_path = "/content/reviews_data_dump.zip"

In [5]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall("data")

os.listdir("data")

['reviews_badminton', 'reviews_tea', 'reviews_tawa']

**Load all csv files**

In [8]:
badminton = pd.read_csv("data/reviews_badminton/data.csv")
tawa = pd.read_csv("data/reviews_tawa/data.csv")
tea = pd.read_csv("data/reviews_tea/data.csv")

**Add product name column**

In [9]:
badminton['Product'] = 'Badminton'
tawa['Product'] = 'Tawa'
tea['Product'] = 'Tea'

**combine all reviews**

In [10]:
df = pd.concat([badminton, tawa, tea], ignore_index=True)
df.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings,Product,Reviewer_Name,...,Up_Votes,Down_Votes,reviewer_name,reviewer_rating,review_title,review_text,place_of_review,Date_of_review,up_votes,Down_votes
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4.0,Badminton,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1.0,Badminton,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1.0,Badminton,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3.0,Badminton,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1.0,Badminton,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20219 entries, 0 to 20218
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Reviewer Name    8508 non-null   object 
 1   Review Title     8508 non-null   object 
 2   Place of Review  8468 non-null   object 
 3   Up Votes         8508 non-null   float64
 4   Down Votes       8508 non-null   float64
 5   Month            8053 non-null   object 
 6   Review text      8510 non-null   object 
 7   Ratings          8518 non-null   float64
 8   Product          20219 non-null  object 
 9   Reviewer_Name    2531 non-null   object 
 10  Reviewer_Rating  2285 non-null   float64
 11  Review_Title     2531 non-null   object 
 12  Review_Text      2531 non-null   object 
 13  Place_of_Review  2531 non-null   object 
 14  Date_of_Review   2531 non-null   object 
 15  Up_Votes         2531 non-null   float64
 16  Down_Votes       2531 non-null   float64
 17  reviewer_nam

In [12]:
df.isnull().sum()

,0
Reviewer Name,11711
Review Title,11711
Place of Review,11751
Up Votes,11711
Down Votes,11711
Month,12166
Review text,11709
Ratings,11701
Product,0
Reviewer_Name,17688


 **create one review_text column**

In [14]:
df['review_text_final'] = (
    df['Review text']
    .combine_first(df['Review_Text'])
    .combine_first(df['review_text'])
    )

**create one rating column**

In [15]:
df['rating_final'] = (
    df['Ratings']
    .combine_first(df['Reviewer_Rating'])
    .combine_first(df['reviewer_rating'])
)

**verify the fix**

In [16]:
df[['review_text_final', 'rating_final', 'Product']].isnull().sum()

,0
review_text_final,8
rating_final,246
Product,0


**drop rows where review or rating is still missing**

In [17]:
df = df.dropna(subset=['review_text_final', 'rating_final'])


**keep only required columns**

In [18]:
df = df[['review_text_final', 'rating_final', 'Product']]

In [19]:
df.columns = ['Review_Text', 'Rating', 'Product']

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19965 entries, 0 to 20218
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review_Text  19965 non-null  object 
 1   Rating       19965 non-null  float64
 2   Product      19965 non-null  object 
dtypes: float64(1), object(2)
memory usage: 623.9+ KB


**checking for any null values**

In [21]:
df.isnull().sum()

,0
Review_Text,0
Rating,0
Product,0


In [22]:
df.head()

,Review_Text,Rating,Product
0,"Nice product, good quality, but price is now r...",4.0,Badminton
1,They didn't supplied Yonex Mavis 350. Outside ...,1.0,Badminton
2,Worst product. Damaged shuttlecocks packed in ...,1.0,Badminton
3,"Quite O. K. , but nowadays the quality of the...",3.0,Badminton
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1.0,Badminton


**sentiment pipeline**

In [24]:
df['Sentiment'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)
df['Sentiment']

,Sentiment
0,1
1,0
2,0
3,0
4,0
...,...
20214,1
20215,1
20216,0
20217,1


**text cleaning & normalization**

In [30]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [34]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^a-zA-Z]', ' ', text)
  words = text.split()
  words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
  return " ".join(words)

df['clean_review'] = df['Review_Text'].apply(clean_text)

**feature extraction (TF-IDF)**

In [38]:
tfidf = TfidfVectorizer(max_features=6000)
X = tfidf.fit_transform(df['clean_review'])
y = df['Sentiment']

**Train_Test Split**

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

**Train Models**
####Logistic Regression

In [43]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

**Evaluate Using F1-Score**

In [45]:
y_pred = model.predict(X_test)

print("F1 Score:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

F1 Score: 0.962846975088968
              precision    recall  f1-score   support

           0       0.91      0.61      0.73       578
           1       0.94      0.99      0.96      3415

    accuracy                           0.93      3993
   macro avg       0.93      0.80      0.85      3993
weighted avg       0.93      0.93      0.93      3993



**Pain Point Analysis (Negative Reviews)**

In [49]:
negative_reviews = df[df['Sentiment'] == 0]

for product in negative_reviews['Product'].unique():
  print(f"\nPain points for {product}")
  text = " ".join(
      negative_reviews[negative_reviews['Product'] == product]['clean_review']
  )
  print(Counter(text.split()).most_common(10))


Pain points for Badminton
[('read', 476), ('shuttle', 426), ('quality', 361), ('product', 325), ('good', 269), ('bad', 217), ('goodread', 186), ('one', 124), ('qualityread', 120), ('worst', 108)]

Pain points for Tawa
[('good', 67), ('read', 49), ('goodread', 41), ('product', 39), ('productread', 28), ('ok', 26), ('nice', 25), ('okread', 25), ('quality', 21), ('handle', 15)]

Pain points for Tea
[('tata', 2751), ('tea', 1834), ('premium', 1834), ('gold', 917), ('v', 917), ('goodread', 917)]


**Save Model & Vectorizer**

In [50]:
import joblib

joblib.dump(model, "sentiment_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']